In [1]:
from pathlib import Path
from time import sleep 

import numpy as np

import roboticstoolbox as rtb
import spatialmath as sm

from manipylator import Robot
from manipylator import utils

[I 04/16/25 02:19:46.056 525] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


# Load Robot
Below we load a URDF file describing Manny the robot manipulator. A URDF is a textual representation of a robot, defining the parameters we need so we can calculate the forward & reverse kinemtics. 

In [2]:
# path = Path('/workspace/robots/robot.urdf')
path = Path('/workspace/robots/robot-ee.urdf')
manny = Robot(path)
print(manny.model)

[Genesis] [02:19:50] [INFO] ╭───────────────────────────────────────────────╮
[Genesis] [02:19:50] [INFO] │┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈ Genesis ┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈┉┈│
[Genesis] [02:19:50] [INFO] ╰───────────────────────────────────────────────╯
[Genesis] [02:19:50] [INFO] Running on [NVIDIA GeForce MX250] with backend gs.cuda. Device memory: 1.95 GB.
[Genesis] [02:19:51] [INFO] 🚀 Genesis initialized. 🔖 version: 0.2.1, 🌱 seed: None, 📏 precision: '32', 🐛 debug: False, 🎨 theme: 'dark'.
[Genesis] [02:19:51] [INFO] Scene <6bee7ae> created.
[Genesis] [02:19:51] [INFO] Adding <gs.RigidEntity>. idx: 0, uid: <aa1d154>, morph: <gs.morphs.Plane>, material: <gs.materials.Rigid>.
[Genesis] [02:19:52] [INFO] Adding <gs.RigidEntity>. idx: 1, uid: <ef3a78d>, morph: <gs.morphs.URDF(file='/workspace/robots/robot-ee.urdf')>, material: <gs.materials.Rigid>.
[Genesis] [02:19:54] [INFO] Building scene <6bee7ae>...
[Genesis] [02:20:20] [INFO] Compiling simulation kernels...
[Genesis] [02:20:29] [INFO] Building visualiz

ERobot: onshape, 6 joints (RRRRRR), dynamics, geometry, collision
┌──────┬──────────────────┬───────┬──────────────────┬────────────────────────────────────────────────┐
│ link │       link       │ joint │      parent      │              ETS: parent to link               │
├──────┼──────────────────┼───────┼──────────────────┼────────────────────────────────────────────────┤
│    0 │ base             │       │ BASE             │ SE3()                                          │
│    1 │ carriage_1       │     0 │ base             │ SE3(0, 0, 0.037; -180°, -0°, 0°) ⊕ Rz(q0)      │
│    2 │ shoulder_lift    │     1 │ carriage_1       │ SE3(-0.065, 0, -0.055; 0°, 90°, -0°) ⊕ Rz(q1)  │
│    3 │ elbow            │     2 │ shoulder_lift    │ SE3(0.35, 0, 0; -180°, -0°, 0°) ⊕ Rz(q2)       │
│    4 │ wrist_carriage_2 │     3 │ elbow            │ SE3(0.32, 0, -0.025; 180°, -0°, 0°) ⊕ Rz(q3)   │
│    5 │ wrist_carriage_3 │     4 │ wrist_carriage_2 │ SE3(0.04, 0, -0.0485; 0°, -90°, -0°) ⊕ Rz(q4)  

# Visualizing Manny
Using the `manny.visualizer.robot` object we can get the properties of it's links. For example, a given link's location in 3D space:

In [3]:
robot = manny.visualizer.robot
# robot

In [4]:
ee = robot.get_link('end_effector')
# ee

In [5]:
ee_initial_pos = ee.get_pos()
ee_initial_pos

tensor([-1.2850e-01, -1.7960e-17,  8.5050e-01], device='cuda:0')

In [6]:
ee_initial_quat = ee.get_quat()
ee_initial_quat

tensor([ 3.5732e-16, -7.0711e-01,  2.7072e-16,  7.0711e-01], device='cuda:0')

And we can verify that for the inverse is also true:

In [7]:
qpos = robot.inverse_kinematics(
    link = ee,
    pos  = ee_initial_pos,
    quat = ee_initial_quat,
)
qpos

[W 04/16/25 02:20:31.995 525] [frontend_ir.cpp:begin_frontend_struct_for_on_external_tensor@1694] ti.loop_config(serialize=True) does not have effect on the struct for. The execution order is not guaranteed.


tensor([0., 0., 0., 0., 0., 0.], device='cuda:0')

I.e. that when setting all the joints to 0 degrees the end effector end up at `ee_initial_pos` and that if the end effector is at `ee_initial_pos` all the joints are at 0 degress.

Similarly, for an arbitrary (reachable) location in 3D space:

In [8]:
qpos = robot.inverse_kinematics(
    link = ee,
    pos  = [-4.8500e-02, -4.7547e-02,  4.0200e-01],
    quat = ee_initial_quat,
)
qpos

tensor([0.0000, 1.3377, 2.0384, 5.2374, 0.0000, 4.5366], device='cuda:0')

# Tracing a Path in R3
Below we calculate a positions in R3 that correspond to a heart shaped curve:

In [78]:
scale = 0.1
offset = 0.3
height = 0.1

shape = utils.parametric_heart_1(np.linspace(0, 2*np.pi, 100))
positions = scale * shape + offset
positions[:,2] = height

positions[:3]

array([[0.3       , 0.3       , 0.1       ],
       [0.30003608, 0.30060238, 0.1       ],
       [0.30028691, 0.30238772, 0.1       ]])

Each row is the (x, y, z) coordinates of a point on the curve. This is useful if we'd like to visualize the expected path of the end effector.

In [80]:
# for x, y, z in positions:
#     manny.visualizer.scene.draw_debug_sphere([x,y,z])
manny.visualizer.scene.draw_debug_spheres(positions)



In [79]:
# manny.visualizer.scene.clear_debug_objects()

Similarly, we can calculate the joint parameters of each position:

In [81]:
magic_quat = [9.6237e-04, -9.1816e-01, -3.9621e-01,  7.0983e-04]

qs = [robot.inverse_kinematics(
        link = ee,
        pos  = pos,
        quat = magic_quat,
        ) for pos in positions]
qs[:5]

[tensor([3.7171, 0.4462, 4.1677, 5.2925, 4.7100, 3.3221], device='cuda:0'),
 tensor([3.7164, 0.4470, 4.1693, 5.2932, 4.7100, 3.3228], device='cuda:0'),
 tensor([3.7145, 0.4497, 4.1743, 5.2955, 4.7100, 3.3247], device='cuda:0'),
 tensor([3.7121, 0.4545, 4.1832, 5.2996, 4.7100, 3.3271], device='cuda:0'),
 tensor([3.7097, 0.4614, 4.1962, 5.3056, 4.7100, 3.3295], device='cuda:0')]

And to visualize the poses Manny would take, we can animate the path:

In [84]:
for pos in qs:
    robot.set_dofs_position(pos)
    # print(robot.get_dofs_position())
    manny.visualizer.scene.step()
    sleep(0.1)

In [71]:
# for pos in qs:
#     dof = robot.inverse_kinematics(
#         link = ee,
#         pos  = pos,
#         quat = magic_quat,
#         )
#     robot.set_dofs_position(dof)
#     # print(robot.get_dofs_position())
#     manny.visualizer.scene.step()
#     sleep(0.1)


KeyboardInterrupt: 